In [ ]:
# Constants

x_c = 0.4
y_c = 2.009
x_p = 0.08
y_p = 2.876
R_0 = 0.16129
C_0 = 0.5

In [ ]:
# Food Chain (3 Species)

# Plants [ R(t) ]
# Herbivores [ C(t) ]
# Carnivores [ P(t) ]

# Solving the coupled differential equations through euler's method
# Assuming R(0) = 1, C(0) = 0.21, P(0) = 0.6

import numpy as np

def dR(R, C, K):
    return ( R*(1-(R/K)) - ((x_c*y_c*C*R)/(R + R_0)) )

def dC(C, R, P):
    return ( x_c*C*( (y_c*R)/(R + R_0) - 1) - ((x_p*y_p*P*C)/(C + C_0)) )

def dP(P, C):
    return ( x_p*P*( ((y_p*C)/(C + C_0)) - 1) )

def EulerMethod(y_0, time, K=0.974):
    R = np.zeros(15000)
    C = np.zeros(15000)
    P = np.zeros(15000)
    R[0] = 1
    C[0] = 0.21
    P[0] = 0.6
    t = np.linspace(0, time, 15000, endpoint=True)
    for i in range(1, len(t)):
        dt = t[i] - t[i-1]
        R[i] = R[i-1] + dR(R[i-1], C[i-1], K)*dt
        C[i] = C[i-1] + dC(C[i-1], R[i-1], P[i-1])*dt
        P[i] = P[i-1] + dP(P[i-1], C[i-1])*dt
    return R, C, P, t

In [ ]:
import matplotlib.pyplot as plt

K = 0.974

R, C, P, t = EulerMethod(1, 3000, K) # K = 0.974

%matplotlib inline
plt.plot(t, P)

# Note that the system suddenly collapses at values of K >= 0.974

In [ ]:
K = 0.96
R, C, P, t = EulerMethod(1, 3000, K) # K = 0.96

In [ ]:
plt.plot(t, P)

In [ ]:
max(P)

In [ ]:
# Use Deep Learning to forecast time-series
# Not using Reservoir Computing, just SimpleRNN

import tensorflow as tf

# Parameters
window_size = 20
batch_size = 32
shuffle_buffer_size = 1000

# Build the Model
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(filters=64, kernel_size=3,
                      strides=1,
                      activation="relu",
                      padding='causal',
                      input_shape=[window_size, 1]),
  tf.keras.layers.SimpleRNN(64, return_sequences=True),
  tf.keras.layers.SimpleRNN(64, return_sequences=True),
  tf.keras.layers.SimpleRNN(64),
  tf.keras.layers.Dense(30, activation="relu"),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(1),
  tf.keras.layers.Lambda(lambda x: x * max(P))
])

# Print the model summary
model.summary()

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    """Generates dataset windows

    Args:
      series (array of float) - contains the values of the time series
      window_size (int) - the number of time steps to include in the feature
      batch_size (int) - the batch size
      shuffle_buffer(int) - buffer size to use for the shuffle method

    Returns:
      dataset (TF Dataset) - TF Dataset containing time windows
    """
  
    # Generate a TF Dataset from the series values
    dataset = tf.data.Dataset.from_tensor_slices(series)
    
    # Window the data but only take those with the specified size
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    
    # Flatten the windows by putting its elements in a single batch
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))

    # Create tuples with features and labels 
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))

    # Shuffle the windows
    dataset = dataset.shuffle(shuffle_buffer)
    
    # Create batches of windows
    dataset = dataset.batch(batch_size).prefetch(1)
    
    return dataset

In [ ]:
def plot_series(time, series, format="-", start=0, end=None):
    """
    Visualizes time series data

    Args:
      time (array of int) - contains the time steps
      series (array of int) - contains the measurements for each time step
      format - line style when plotting the graph
      start - first time step to plot
      end - last time step to plot
    """

    # Setup dimensions of the graph figure
    plt.figure(figsize=(10, 6))
    
    if type(series) is tuple:

      for series_num in series:
        # Plot the time series data
        plt.plot(time[start:end], series_num[start:end], format)

    else:
      # Plot the time series data
      plt.plot(time[start:end], series[start:end], format)

    # Label the x-axis
    plt.xlabel("Time")

    # Label the y-axis
    plt.ylabel("Value")

    # Overlay a grid on the graph
    plt.grid(True)

    # Draw the graph on screen
    plt.show()

In [ ]:
# Define the split time
split_time = 13000

# Get the train set 
time_train = t[:split_time]
x_train = P[:split_time]

# Get the validation set
time_valid = t[split_time:]
x_valid = P[split_time:]

In [ ]:
# Generate the dataset windows
dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

In [ ]:
# Get initial weights
init_weights = model.get_weights()

In [ ]:
# Set the learning rate scheduler
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 20))

# Initialize the optimizer
optimizer = tf.keras.optimizers.SGD(momentum=0.9)

# Set the training parameters
model.compile(loss=tf.keras.losses.Huber(), optimizer=optimizer)

# Train the model
history = model.fit(dataset, epochs=150, callbacks=[lr_schedule])

In [ ]:
# Define the learning rate array
lrs = 1e-8 * (10 ** (np.arange(150) / 20))

# Set the figure size
plt.figure(figsize=(10, 6))

# Set the grid
plt.grid(True)

# Plot the loss in log scale
plt.semilogx(lrs, history.history["loss"])

# Increase the tickmarks size
plt.tick_params('both', length=10, width=1, which='both')

# Set the plot boundaries
plt.axis([1e-8, 1, 0, 0.006])

In [ ]:
# Reset states generated by Keras
tf.keras.backend.clear_session()

# Reset the weights
model.set_weights(init_weights)

In [ ]:
# Set the learning rate
learning_rate = 3e-4

# Set the optimizer 
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)

# Set the training parameters
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])

In [ ]:
# Train the model
history = model.fit(dataset,epochs=100)

In [ ]:
# Initialize a list
forecast = []

# Reduce the original series
forecast_series = P[split_time - window_size:]

# Use the model to predict data points per window size
for time in range(len(forecast_series) - window_size):
  forecast.append(model.predict(forecast_series[time:time + window_size][np.newaxis]))

# Convert to a numpy array and drop single dimensional axes
results = np.array(forecast).squeeze()

# Plot the results
plot_series(time_valid, (x_valid, results))

In [ ]:
# That was a very good prediction
# Can we do something about the bifurcation parameter ?
# Can we use it on the model as input ?
# Yes, of course

tf.keras.backend.clear_session()

# Time series input
inputA = tf.keras.layers.Input(shape=(window_size, 1))
conv1d_layer = tf.keras.layers.Conv1D(filters=64, kernel_size=3,
                                       strides=1,
                                       activation="relu",
                                       padding='causal')
simple_rnn1_layer = tf.keras.layers.SimpleRNN(64, return_sequences=True)
simple_rnn2_layer = tf.keras.layers.SimpleRNN(64, return_sequences=True)
simple_rnn3_layer = tf.keras.layers.SimpleRNN(64)

# Bifurcation parameter input
inputB = tf.keras.layers.Input(shape=(1,))

# Fully connected layer
dense1_layer = tf.keras.layers.Dense(30, activation="relu")
dense2_layer = tf.keras.layers.Dense(10, activation="relu")
dense3_layer = tf.keras.layers.Dense(1)

# Model
x = conv1d_layer(inputA)
x = simple_rnn1_layer(x)
x = simple_rnn2_layer(x)
x = simple_rnn3_layer(x)

# Concatenate the output of the RNN with the bifurcation parameter
x = tf.keras.layers.Concatenate()([x, inputB])

# Dense layers
x = dense1_layer(x)
x = dense2_layer(x)
x = dense3_layer(x)

# Final model
model = tf.keras.models.Model(inputs=[inputA, inputB], outputs=x)

learning_rate = 3e-4

optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)

model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer, K):
    """Generates dataset windows

    Args:
      series (array of float) - contains the values of the time series
      window_size (int) - the number of time steps to include in the feature
      batch_size (int) - the batch size
      shuffle_buffer(int) - buffer size to use for the shuffle method

    Returns:
      dataset (TF Dataset) - TF Dataset containing time windows
    """
  
    # Generate a TF Dataset from the series values
    dataset = tf.data.Dataset.from_tensor_slices(series)
    
    # Window the data but only take those with the specified size
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    
    # Flatten the windows by putting its elements in a single batch
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))

    # Create tuples with features and labels 
    dataset = dataset.map(lambda window: ((window[:-1], K), window[-1]))

    # Shuffle the windows
    dataset = dataset.shuffle(shuffle_buffer)
    
    # Create batches of windows
    dataset = dataset.batch(batch_size).prefetch(1)
    
    return dataset

In [ ]:
dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size, K)

In [ ]:
model.fit(dataset, epochs=100)

In [ ]:
# Initialize a list
forecast = []

# Reduce the original series
forecast_series = P[split_time - window_size:]

# Use the model to predict data points per window size
for time in range(len(forecast_series) - window_size):
  forecast.append(model.predict([forecast_series[time:time + window_size][np.newaxis], np.array([[K]])]))

# Convert to a numpy array and drop single dimensional axes
results = np.array(forecast).squeeze()

# Plot the results
plot_series(time_valid, (x_valid, results))

In [ ]:
# Now we can use the bifurcation parameter as input to the model
# We can vary the bifurcation parameter and see how the model behaves

K = 0.97
R, C, P, t = EulerMethod(1, 3000, K)
plt.plot(t, P)

In [ ]:
def train_model(P, t, K):
    split_time = 13000
    time_train = t[:split_time]
    x_train = P[:split_time]
    time_valid = t[split_time:]
    x_valid = P[split_time:]
    dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size, K)
    model.fit(dataset, epochs=100)
    forecast = []
    forecast_series = P[split_time - window_size:]
    for time in range(len(forecast_series) - window_size):
        forecast.append(model.predict([forecast_series[time:time + window_size][np.newaxis], np.array([[K]])]))
    results = np.array(forecast).squeeze()
    plot_series(time_valid, (x_valid, results))
    

In [ ]:
train_model(P, t, K)

In [ ]:
K = 0.971
R, C, P, t = EulerMethod(1, 3000, K)
plt.plot(t, P)

In [ ]:
train_model(P, t, K)

In [ ]:
K = 0.972
R, C, P, t = EulerMethod(1, 3000, K)
plt.plot(t, P)

In [ ]:
train_model(P, t, K)

In [ ]:
K = 0.973
R, C, P, t = EulerMethod(1, 3000, K)
plt.plot(t, P)

In [ ]:
train_model(P, t, K)

In [ ]:
K = 0.974
R, C, P, t = EulerMethod(1, 3000, K)
plt.plot(t, P)

In [ ]:
train_model(P, t, K)

In [ ]:
K = 0.975
R, C, P, t = EulerMethod(1, 3000, K)
plt.plot(t, P)

In [ ]:
train_model(P, t, K)

In [ ]:
K = 0.976
R, C, P, t = EulerMethod(1, 3000, K)
plt.plot(t, P)

In [ ]:
train_model(P, t, K)

In [ ]:
K = 0.977
R, C, P, t = EulerMethod(1, 3000, K)
plt.plot(t, P)

In [ ]:
train_model(P, t, K)

In [ ]:
K = 0.978
R, C, P, t = EulerMethod(1, 3000, K)
plt.plot(t, P)

In [ ]:
train_model(P, t, K)

In [ ]:
# Let's try with unseen K values
# First K = 0,99

# What we expect
K = 0.99
R, C, P, t = EulerMethod(1, 3000, K)
plt.plot(t, P)

In [ ]:
# What the model predict

forecast = []
forecast_series = P.copy()
for time in range(len(forecast_series) - window_size):
  forecast.append(model.predict([forecast_series[time:time + window_size][np.newaxis], np.array([[K]])]))
results = np.array(forecast).squeeze()
plot_series(t[window_size:], results)

In [ ]:
# Comparison

plot_series(t[window_size:], (P[window_size:], results))

In [ ]:
# Second K = 0,94

# What we expect
K = 0.94
R, C, P, t = EulerMethod(1, 3000, K)
plt.plot(t, P)

In [ ]:
# What the model predict

forecast = []
forecast_series = P.copy()
for time in range(len(forecast_series) - window_size):
  forecast.append(model.predict([forecast_series[time:time + window_size][np.newaxis], np.array([[K]])]))
results = np.array(forecast).squeeze()
plot_series(t[window_size:], results)

In [ ]:
# Comparison

plot_series(t[window_size:], (P[window_size:], results))

In [ ]:
# Third K = 0,80

# What we expect
K = 0.80
R, C, P, t = EulerMethod(1, 3000, K)
plt.plot(t, P)

In [ ]:
# What the model predict

forecast = []
forecast_series = P.copy()
for time in range(len(forecast_series) - window_size):
  forecast.append(model.predict([forecast_series[time:time + window_size][np.newaxis], np.array([[K]])]))
results = np.array(forecast).squeeze()
plot_series(t[window_size:], results)

In [ ]:
# Comparison

plot_series(t[window_size:], (P[window_size:], results))

In [ ]:
K = 0.80
R, C, P, t = EulerMethod(1, 3000, K)
plt.plot(t, P)

In [ ]:
train_model(P, t, K)

In [ ]:
# Suggestion: train more with K values below 0.9
# Save
#model.save('model.ckpt')

In [ ]:
# Load

import tensorflow as tf

model = tf.keras.models.load_model('model.ckpt')

In [ ]:
import matplotlib.pyplot as plt

window_size = 20
batch_size = 32
shuffle_buffer_size = 1000